# Notebook to compute requested anomaly for WP1 TiPACCs

## Modules

In [ ]:
import numpy as np
import os
import copy
from datetime import datetime

# data
import netCDF4 as nc                             # read netcdf
import xarray as xr                              # write netcdf

## Define data

In [ ]:
# tag
ctag='y2029.10y'

# case
cconfcase_ref='eORCA025.L121-OPM021'
cconfcase_ano='eORCA025.L121-OPM020'

# input file ISMIP6_eORCA025.L121-OPM021_y1999.10y_WP1_TiPACCs.nc
cfile_ref='ISMIP6_{}_{}_WP1_TiPACCS.nc'.format(cconfcase_ref,ctag)
cfile_ano='ISMIP6_{}_{}_WP1_TiPACCS.nc'.format(cconfcase_ano,ctag)

# output file
cf_out='ISMIP6_{}_{}_anomaly_WP1_TiPACCs.nc'.format(cconfcase_ano,ctag)

## Load data

In [ ]:
# dataset in
ds_ref=xr.open_dataset(cfile_ref)
ds_ano=xr.open_dataset(cfile_ano)

# data set out
ds_out=xr.open_dataset(cfile_ano)

In [ ]:
ds_out

## Compute anomaly

In [ ]:
for cvar in ['thetao','so','tob','sob','ficeshelf']:
    dataout=ds_ano[cvar].to_masked_array() - ds_ref[cvar].to_masked_array()
    minval=np.min(dataout)
    maxval=np.max(dataout)
    clong_name=ds_ano[cvar].attrs['long_name']+' anomaly'
    ds_out[cvar].values = np.ma.masked_invalid(dataout).filled(fill_value=-9999.99)
    ds_out[cvar].attrs['valid_min']=minval
    ds_out[cvar].attrs['valid_max']=maxval
    ds_out[cvar].attrs['long_name']=clong_name
    ds_out=ds_out.rename({cvar : 'd_'+cvar})

## write file

In [ ]:
# global attributes
dgatt = {'description' : 'The above anomaly file is the differences between a tipped state and a reference state',
         'source' :'{} - {} NEMO simulations'.format(cconfcase_ano,cconfcase_ref),
         'model time' : '10 years average starting 01/01/1999',
         'grid'   : 'ISMIP6',
         'contact':'P. Mathiot (IGE)',
         'creation date':'{}'.format(datetime.now()),
        }

print('    write global att')
ds_out.attrs = dgatt

print('    write netcdf ...')
ds_out.to_netcdf(cf_out)
print('--------------------------------')